# 웹크롤링

In [1]:
import pandas as pd
import time
from PIL import Image
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

In [2]:
url = 'https://comic.naver.com/webtoon'
driver = webdriver.Chrome()
driver.get(url)
#time.sleep(3)
driver.implicitly_wait(time_to_wait=10)

### 웹툰별 항목 확인

제목, 업로드 요일, 전처리한 썸네일 파일명, 상세페이지 링크

In [3]:
import re
def check_title_validation(r, string): 
    if r.search(string) is not None:
        return False
    return True

def modify_title(r, is_valid, string):
    if is_valid:
        return string
    return r.sub('', string)

In [5]:
r = re.compile("[^a-zA-Z0-9ㄱ-ㅎ가-힣+]")

attributes = ["제목", "요일", "썸네일_파일명", "링크", "장르", "태그"]
df_webtoon = pd.DataFrame(columns=attributes)

week_lut = "월화수목금토일"
mod_titles = []
counts_for_week = []
title_links = []
for day in range(1, 8):
    print(week_lut[day - 1])
    lists = driver.find_elements(By.XPATH, f'//*[@id="container"]/div[3]/div[2]/div[{month}]/ul/li')
    counts_for_week.append(len(lists))
    for i in range(1, len(lists) + 1):
        link = driver.find_element(By.XPATH, f'//*[@id="container"]/div[3]/div[2]/div[{month}]/ul/li[{i}]/div/a').get_attribute("href")
        title = driver.find_element(By.XPATH, f'//*[@id="container"]/div[3]/div[2]/div[{month}]/ul/li[{i}]/div/a/span/span').text
        
        is_valid = check_title_validation(r, title)
        mod_title = modify_title(r, is_valid, title)
        data_rows = [title, week_lut[day - 1], mod_title, link]
        df_webtoon.loc[len(df_webtoon)] = data_rows + [None] * 2
        mod_titles.append(mod_title)
        print(title, mod_title)
    print()

월
뷰티풀 군바리 뷰티풀군바리
오늘은 나랑 만나 오늘은나랑만나
신의 탑 신의탑
참교육 참교육
백수세끼 백수세끼
퀘스트지상주의 퀘스트지상주의
윈드브레이커 윈드브레이커
장씨세가 호위무사 장씨세가호위무사
소녀의 세계 소녀의세계
버림받은 왕녀의 은밀한 침실 버림받은왕녀의은밀한침실
신화급 귀속 아이템을 손에 넣었다 신화급귀속아이템을손에넣었다
앵무살수 앵무살수
2023 루키 단편선 2023루키단편선
별난식당 별난식당
절대검감 절대검감
퍼니게임 퍼니게임
우아한 욕망 우아한욕망
만렙돌파 만렙돌파
잔불의 기사 잔불의기사
물어보는 사이 물어보는사이
ㅋㅋ단편.zip ㅋㅋ단편zip
개같이 탈출 개같이탈출
순정말고 순종 순정말고순종
미친 재능의 플레이어 미친재능의플레이어
리턴 투 플레이어 리턴투플레이어
악당 가족이 독립을 반대한다 악당가족이독립을반대한다
좀비묵시록 82-08 좀비묵시록8208
천재 타자가 강속구를 숨김 천재타자가강속구를숨김
히어로메이커 히어로메이커
신군 신군
루루라라 우리네 인생 루루라라우리네인생
제왕 제왕
칼가는 소녀 칼가는소녀
날 먹는 건 금지양! 날먹는건금지양
결혼생활 그림일기 결혼생활그림일기
더블클릭 더블클릭
시한부의 아이까지 뺏으려 합니다 시한부의아이까지뺏으려합니다
아슈타르테 아슈타르테
제국 제일의 상속녀가 되었습니다 제국제일의상속녀가되었습니다
오빠집이 비어서 오빠집이비어서
파운더 파운더
어린이집 다니는 구나 어린이집다니는구나
달로 만든 아이 달로만든아이
인섹터 인섹터
팔이피플 팔이피플
꿈의 기업 꿈의기업
서브 남주가 파업하면 생기는 일 서브남주가파업하면생기는일
쥴리에타의 드레스 업 쥴리에타의드레스업
엘프 엘프
파견체 파견체
사이다걸 사이다걸
라서드 라서드
입술이 예쁜 남자 입술이예쁜남자
악녀교실 악녀교실
피폐물의 해피엔딩을 위하여 피폐물의해피엔딩을위하여
영앤리치가 아니야! 영앤리치가아니야
컨트롤X 컨트롤X
마이너스의 손 마이너스의손
랭커 랭커
악당과 악당이 만나면 악당과악당이만나면
하루의 하루 하루의하루
백호랑 백호랑
너를 돌려차는 방법 너를돌려차는방법
최

In [6]:
# 요일별 웹툰 개수
print(counts_for_week)

[92, 94, 93, 98, 91, 96, 89]


### 웹툰별 썸네일 저장

In [ ]:
# 썸네일 링크 저장
# 썸네일 크롤링을 위한 focus 변경
action = ActionChains(driver)

idx = 0  # ~ len(df_webtoon)
thumbnail_src = []
for day, count in enumerate(counts_for_week):
    img = driver.find_element(By.XPATH, f'//*[@id="container"]/div[3]/div[2]/div[{day + 1}]/ul/li[1]/a/div/img')
    action.move_to_element(img).perform()
    for i in range(1, counts_for_week[day] + 1):
        title = mod_titles[idx]
        idx += 1
        img_element = driver.find_element(By.XPATH, f'//*[@id="container"]/div[3]/div[2]/div[{day + 1}]/ul/li[{i}]/a/div/img')
        img_src = img_element.get_attribute('src')
        img_alt = img_element.get_attribute('alt')
        print(img_src)
        thumbnail_src.append(img_src)
        action.move_to_element(img_element).perform()

driver.close()

In [ ]:
# 썸네일 스크린샷
for idx, (src, t) in enumerate(zip(thumbnail_src, mod_titles)):
    thumbnail = webdriver.Chrome()
    thumbnail.set_window_size(480, 760)
    thumbnail.get(src)
    time.sleep(10)
    thumbnail.save_screenshot(f'thumbnails/tmp_thumbnails/tmp_{t}.png')
    thumbnail.close()
    if idx % 50 == 0:
        print(f"{idx + 1}번째 파일 완료")
else:
    print(f"{idx + 1}번째 파일 완료") 

In [13]:
# 썸네일 crop
for idx, (src, t) in enumerate(zip(thumbnail_src, mod_titles)):
    image1 = Image.open(f"thumbnails/tmp_thumbnails/tmp_{t}.png")
    croppedImage = image1.crop((11, 0, 490, 622))
    croppedImage.save(f"thumbnails/{t}.png")
    if idx % 50 == 0:
        print(f"{idx + 1}번째 파일 완료")
else:
    print(f"{idx + 1}번째 파일 완료") 

### 웹툰별 장르 및 태그 확인

In [ ]:
links = list(df_webtoon['링크'])

genre = []
tags = []
error = []
for idx, url in enumerate(links):
    toon = webdriver.Chrome()
    toon.get(url)
    #time.sleep(3)
    toon.implicitly_wait(time_to_wait=10)
    try:
        whole_tags = toon.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[2]/div/div').text.split('\n')
    except:
        whole_tags = ['']
        error.append((idx, url))
    genre.append(whole_tags[0])
    if len(whole_tags) > 1:
        tags.append(whole_tags[1:])
    else:
        tags.append([])
    if idx % 50 == 0:
        print(idx + 1, len(genre))

In [20]:
print(len(df_webtoon), len(genre), len(tags))

622 622 622


In [22]:
# 로그인 필요한 웹툰은 별도 처리
for idx, url in error:
    toon = webdriver.Chrome()
    toon.get(url)
    #time.sleep(3)
    input("로그인 후 Enter를 입력하세요.")
    whole_tags = toon.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[2]/div/div').text.split('\n')
    genre[idx] = whole_tags[0]
    if len(whole_tags) > 1:
        tags[idx] = whole_tags[1:]

In [23]:
df_webtoon['장르'] = genre
df_webtoon['태그'] = tags
df_webtoon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622 entries, 0 to 621
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   제목       622 non-null    object
 1   요일       622 non-null    object
 2   썸네일_파일명  622 non-null    object
 3   링크       622 non-null    object
 4   장르       622 non-null    object
 5   태그       622 non-null    object
dtypes: object(6)
memory usage: 29.3+ KB


### 데이터 저장

In [41]:
df_webtoon.to_excel('webtoon_summary.xlsx')

### 요일별 장르 개수 확인

In [43]:
genre_set = set(genre)
statistics = {g: {d: 0 for d in week_lut} for g in list(genre_set)} 
print(statistics)
for i in range(len(df_webtoon)):
    day = df_webtoon.loc[i, '요일'][0]
    gen = df_webtoon.loc[i, '장르']
    if gen == '':
        continue
    statistics[gen][day] += 1

print(statistics)

{'#액션': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#개그': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#스릴러': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#일상': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#감성': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#스포츠': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#드라마': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#무협/사극': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#판타지': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}, '#로맨스': {'월': 0, '화': 0, '수': 0, '목': 0, '금': 0, '토': 0, '일': 0}}
{'#액션': {'월': 7, '화': 10, '수': 12, '목': 9, '금': 10, '토': 7, '일': 8}, '#개그': {'월': 2, '화': 4, '수': 3, '목': 4, '금': 1, '토': 2, '일': 4}, '#스릴러': {'월': 3, '화': 6, '수': 10, '목': 5, '금': 11, '토': 6, '일': 4}, '#일상': {'월': 3, '화': 6, '수': 4, '목': 3, '금': 3, '토': 3, '일': 2}, '#감성': {'월': 1, '화': 2, '수': 0, '목': 0, '금': 1, '토': 0, '일': 0}, '#스포츠': {